In [7]:
from cesm_le2_recipe import recipe

* Add pr, add directory cesm2-le in staged-recipes/recipes
## Setup to run the recipe

Set the source and target paths

In [8]:
from pathlib import Path

# Where the files are stored locally
direct = '/glade/campaign/cgd/cesm/CESM2-LE/timeseries/atm/proc/tseries/month_1/T'

# Where to write the zarr stores to
target_dir = '/glade/scratch/mgrover/cesm2-le'

# Where to cache metadata
meta_dir = '/glade/scratch/mgrover/cesm2-le-metadata'

In [9]:
from fsspec.implementations.local import LocalFileSystem
from pangeo_forge_recipes.storage import FSSpecTarget, MetadataTarget

fs_local = LocalFileSystem()

cache_dir = tempfile.TemporaryDirectory()
cache_target = FSSpecTarget(fs_local, direct)

target = FSSpecTarget(fs_local, target_dir)

meta_store = MetadataTarget(fs_local, meta_dir)

recipe.input_cache = cache_target
recipe.target = target
recipe.metadata_cache = meta_store

cache_target.root_path, target.root_path, meta_store.root_path

('/glade/campaign/cgd/cesm/CESM2-LE/timeseries/atm/proc/tseries/month_1/T',
 '/glade/scratch/mgrover/cesm2-le',
 '/glade/scratch/mgrover/cesm2-le-metadata')

### Cache metadata

In [10]:
for input_name in recipe.iter_inputs():
    recipe.cache_input_metadata(input_name)

### Prepare the target
Basically starts the zarr store - writes the first time step

In [11]:
recipe.prepare_target()

## Check to see if the target is there

In [15]:
import zarr

# is it there?
zgroup = zarr.open(target_dir)
print(zgroup.tree())

/
 ├── P0 () float64
 ├── T (1980, 32, 192, 288) float32
 ├── ch4vmr (1980,) float64
 ├── co2vmr (1980,) float64
 ├── date (1980,) float64
 ├── date_written (1980,) |S8
 ├── datesec (1980,) float64
 ├── f11vmr (1980,) float64
 ├── f12vmr (1980,) float64
 ├── gw (192,) float64
 ├── hyai (33,) float64
 ├── hyam (32,) float64
 ├── hybi (33,) float64
 ├── hybm (32,) float64
 ├── ilev (33,) float64
 ├── lat (192,) float64
 ├── lev (32,) float64
 ├── lon (288,) float64
 ├── mdt () float64
 ├── n2ovmr (1980,) float64
 ├── nbdate () float64
 ├── nbsec () float64
 ├── ndbase () float64
 ├── ndcur (1980,) float64
 ├── nsbase () float64
 ├── nscur (1980,) float64
 ├── nsteph (1980,) float64
 ├── sol_tsi (1980,) float64
 ├── time (1980,) int64
 ├── time_bnds (1980, 2) int64
 ├── time_written (1980,) |S8
 ├── zlon (1,) float64
 └── zlon_bnds (1, 2) float64


## Take a look at the file

In [18]:
import xarray as xr
# Check to see if it worked:
ds = xr.open_zarr(target_dir)
print(ds)

<xarray.Dataset>
Dimensions:       (ilev: 33, lat: 192, lev: 32, lon: 288, nbnd: 2, time: 1980, zlon: 1)
Coordinates:
  * ilev          (ilev) float64 2.255 5.032 10.16 18.56 ... 967.5 985.1 1e+03
  * lat           (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lev           (lev) float64 3.643 7.595 14.36 24.61 ... 957.5 976.3 992.6
  * lon           (lon) float64 0.0 1.25 2.5 3.75 ... 355.0 356.2 357.5 358.8
  * time          (time) object 2010-02-01 00:00:00 ... 2010-02-01 00:00:00
  * zlon          (zlon) float64 0.0
Dimensions without coordinates: nbnd
Data variables: (12/27)
    P0            float64 ...
    T             (time, lev, lat, lon) float32 dask.array<chunksize=(14, 32, 192, 288), meta=np.ndarray>
    ch4vmr        (time) float64 dask.array<chunksize=(14,), meta=np.ndarray>
    co2vmr        (time) float64 dask.array<chunksize=(14,), meta=np.ndarray>
    date          (time) float64 dask.array<chunksize=(14,), meta=np.ndarray>
    date_written  (time

In [21]:
ds['T']

,Array,Chunk
Bytes,13.05 GiB,94.50 MiB
Shape,"(1980, 32, 192, 288)","(14, 32, 192, 288)"
Count,143 Tasks,142 Chunks
Type,float32,numpy.ndarray
